# Test the pipeline with the Hungarian Kalman Filter (WKF).

In [1]:
import os
import numpy as np
import torch
import cv2
from torch.utils.data import DataLoader
import sys
import matplotlib.pyplot as plt

# Get the path of the project root
project_root = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))

# Add the project root to sys.path if it is not already in sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

# Import the ImageSequenceDataset and create_dataloader from the pipeline module
from seismogram_extraction.pipeline import evaluate_filter, create_dataloader
from seismogram_extraction.filters.hungarian_kalman_filter_bis import HungarianKalmanFilterBis

import torch.multiprocessing as mp

In [2]:
mp.set_start_method('spawn')  # or 'forkserver' if available

### Parameters
Dt = 0.005 # np.linspace(0.5, 5, 2) # Time step related to the state transition matrix A, ! different than sampling rate dt of signal s

# Assuming no process noise
sigma_p = 1 # np.linspace(1e-2, 2, 2) 
sigma_v = 0.8 # np.linspace(1e-2, 2, 2)
sigma_a = 0.8 # np.linspace(1e-2, 2, 2)

# Assuming no measurement noise
sigma_z = 0.5 # np.linspace(1e-6, 1, 5)

source = r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction\data\sines\overlap_0-0"
###

A = np.array([[1, Dt, Dt],
              [0, 1, Dt], 
              [0, 0, 1]]).astype(np.float64)
                        
H = np.array([[1, 0, 0]]).astype(np.float64)

Q = np.array([[sigma_p**2, 0, 0],
              [0, sigma_v**2, 0],
              [0, 0, sigma_a**2]])

R = np.array([[sigma_z**2]])

# Initial state covariance given all_x_0
P = np.zeros((3, 3))
P[1, 1] = 10
P[2, 2] = 10

image_folder_path = source + r"\signals"
GTs_folder_path = source + r"\ground_truth"
    
dataloader = create_dataloader(image_folder_path, GTs_folder_path, batch_size=4)

output_folder_path = source + r"\processed_HKF_bis"
os.makedirs(output_folder_path, exist_ok=True)
    
evaluate_filter(image_folder_path, GTs_folder_path, output_folder_path, HungarianKalmanFilterBis(A, H, Q, R))

Batches:  33%|███▎      | 1/3 [00:11<00:22, 11.44s/it]


MSE of the batch 0: 932.1687416848017


Batches:  67%|██████▋   | 2/3 [00:15<00:06,  6.99s/it]


MSE of the batch 1: 4325.733234072735


Batches: 100%|██████████| 3/3 [00:17<00:00,  4.69s/it]


MSE of the batch 2: 1505.9522413148165


Batches: 100%|██████████| 3/3 [00:18<00:00,  6.11s/it]

Total Average MSE: 2254.618072357451
Std: 1483.115051937894


(2254.618072357451, 2254.618072357451, 1483.115051937894)